In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv


In [2]:
from zipfile import ZipFile
from glob import glob
from PIL import Image
from rich.progress import track
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [3]:
with ZipFile('/kaggle/input/dogs-vs-cats/train.zip', 'r') as f:
    f.extractall(path='/train')

In [4]:
dogs = glob('/train/train/dog*')
cats = glob('/train/train/cat*')

y = np.array([0] * len(dogs) + [1] * len(cats))
x = []

for location in track(dogs+cats):
    x.append(np.array(Image.open(location).resize((70,70))))
x = np.array(x)

x.shape, y.shape

Output()

((25000, 70, 70, 3), (25000,))

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [6]:
for i in [x_train, x_test, y_train, y_test]:
    print(i.shape)

(17500, 70, 70, 3)
(7500, 70, 70, 3)
(17500,)
(7500,)


In [7]:
model = Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(70,70,3)),
    MaxPool2D(2,2),
    Conv2D(16,(2,2),activation='relu'),
    MaxPool2D(2,2),
    Flatten(),
    Dense(100,activation='relu'),
    Dense(10,activation='relu'),
    Dense(2,activation='softmax')
])

2023-01-29 10:10:19.824356: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [9]:
model.fit(x_train,y_train,epochs=20)

2023-01-29 10:10:20.050702: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
547/547 [==============================] - 20s 36ms/step - loss: 0.9848 - accuracy: 0.5742
Epoch 2/20
547/547 [==============================] - 20s 36ms/step - loss: 0.6362 - accuracy: 0.6389
Epoch 3/20
547/547 [==============================] - 20s 36ms/step - loss: 0.5925 - accuracy: 0.6800
Epoch 4/20
547/547 [==============================] - 19s 35ms/step - loss: 0.5344 - accuracy: 0.7239
Epoch 5/20
547/547 [==============================] - 20s 36ms/step - loss: 0.4860 - accuracy: 0.7527
Epoch 6/20
547/547 [==============================] - 20s 37ms/step - loss: 0.4086 - accuracy: 0.8059
Epoch 7/20
547/547 [==============================] - 20s 36ms/step - loss: 0.3340 - accuracy: 0.8464
Epoch 8/20
547/547 [==============================] - 20s 36ms/step - loss: 0.2797 - accuracy: 0.8766
Epoch 9/20
547/547 [==============================] - 19s 35ms/step - loss: 0.2284 - accuracy: 0.9079
Epoch 10/20
547/547 [==============================] - 20s 36ms/step - loss: 0.167

In [11]:
model.evaluate(x_test, y_test)

235/235 [==============================] - 4s 15ms/step - loss: 2.1225 - accuracy: 0.6480


[2.122535228729248, 0.6480000019073486]